## Homework

> Note: sometimes your answer doesn't match one of the options exactly. 
> That's fine. 
> Select the option that's closest to your solution.


### Dataset

In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv).

Or you can do it with `wget`:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
```

In this dataset our desired target for classification task will be `converted` variable - has the client signed up to the platform or not. 

### Data preparation

* Check if the missing values are presented in the features.
* If there are missing values:
    * For caterogiral features, replace them with 'NA'
    * For numerical features, replace with with 0.0 

In [91]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from IPython.display import display

import matplotlib.pyplot as plt

In [2]:
file = 'course_lead_scoring.csv'
if os.path.exists(file):
    pass
else:
    data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
    !wget $data -O course_lead_scoring.csv  
    

In [3]:
df = pd.read_csv('course_lead_scoring.csv')
df.iloc[:10]

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
5,events,manufacturing,1,59904.0,NaN,africa,6,0.83,1
6,social_media,technology,0,51283.0,NaN,middle_east,2,0.57,0
7,social_media,NaN,5,62975.0,student,europe,4,0.62,1
8,referral,healthcare,4,38648.0,unemployed,south_america,2,0.86,1
9,paid_ads,other,3,59866.0,student,australia,3,0.43,1


In [4]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)
categorical_columns

['lead_source', 'industry', 'employment_status', 'location']

In [7]:
numerical_columns = list(df.dtypes[df.dtypes != 'object'].index)
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [8]:
df[categorical_columns].isnull().sum()

lead_source          128
industry             134
employment_status    100
location              63
dtype: int64

In [9]:
df[numerical_columns].isnull()

,number_of_courses_viewed,annual_income,interaction_count,lead_score,converted
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
1457,False,True,False,False,False
1458,False,False,False,False,False
1459,False,False,False,False,False
1460,False,False,False,False,False


In [10]:
df[df[numerical_columns].annual_income.isnull()].index

Index([  14,   58,   70,   72,   79,   96,  106,  127,  131,  133,
       ...
       1412, 1414, 1419, 1424, 1426, 1428, 1432, 1437, 1455, 1457],
      dtype='int64', length=181)

In [11]:
df.iloc[14]

lead_source                    paid_ads
industry                     healthcare
number_of_courses_viewed              3
annual_income                       NaN
employment_status            unemployed
location                    middle_east
interaction_count                     4
lead_score                         0.75
converted                             1
Name: 14, dtype: object

In [12]:
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [13]:
df[categorical_columns].isnull().sum()

lead_source          0
industry             0
employment_status    0
location             0
dtype: int64

In [14]:
df[numerical_columns] = df[numerical_columns].fillna(0.0)

In [15]:
df[numerical_columns].isnull().sum()

number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

In [16]:
df.iloc[:10]

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
5,events,manufacturing,1,59904.0,NA,africa,6,0.83,1
6,social_media,technology,0,51283.0,NA,middle_east,2,0.57,0
7,social_media,NA,5,62975.0,student,europe,4,0.62,1
8,referral,healthcare,4,38648.0,unemployed,south_america,2,0.86,1
9,paid_ads,other,3,59866.0,student,australia,3,0.43,1


In [17]:
df[numerical_columns].iloc[14]

number_of_courses_viewed    3.00
annual_income               0.00
interaction_count           4.00
lead_score                  0.75
converted                   1.00
Name: 14, dtype: float64

### Question 1

What is the most frequent observation (mode) for the column `industry`?

- `NA`
- `technology`
- `healthcare`
- `retail`

In [18]:
df.industry.describe()

count       1462
unique         8
top       retail
freq         203
Name: industry, dtype: object

#### The most frequent observation (mode) for column industry is `RETAIL`.

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

- `interaction_count` and `lead_score`                      0.025393  0.009888
- `number_of_courses_viewed` and `lead_score`               0.009427 -0.004879
- `number_of_courses_viewed` and `interaction_count`       -0.044381 -0.023565
- `annual_income` and `interaction_count`                   0.011959  0.027036	

Only consider the pairs above when answering this question.


In [19]:
categorical = categorical_columns

In [20]:
numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score',
 'converted']

In [21]:
numerical = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']
numerical

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [22]:
df[numerical].corrwith(df.lead_score)

number_of_courses_viewed   -0.004879
annual_income               0.015610
interaction_count           0.009888
lead_score                  1.000000
dtype: float64

In [23]:
df[numerical].corrwith(df.lead_score)

number_of_courses_viewed   -0.004879
annual_income               0.015610
interaction_count           0.009888
lead_score                  1.000000
dtype: float64

In [24]:
df[numerical].corrwith(df.interaction_count)


number_of_courses_viewed   -0.023565
annual_income               0.027036
interaction_count           1.000000
lead_score                  0.009888
dtype: float64

In [53]:
corr_matrix = df[numerical].corr()

In [54]:
corr_matrix

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [55]:
max_corr = corr_matrix.unstack().sort_values(ascending=False)
max_corr = max_corr[max_corr < 1]
top_features = max_corr.idxmax()
print(f"The two features with the highest correlation are: {top_features}")

The two features with the highest correlation are: ('annual_income', 'interaction_count')


In [56]:
max_corr

annual_income             interaction_count           0.027036
interaction_count         annual_income               0.027036
lead_score                annual_income               0.015610
annual_income             lead_score                  0.015610
lead_score                interaction_count           0.009888
interaction_count         lead_score                  0.009888
annual_income             number_of_courses_viewed    0.009770
number_of_courses_viewed  annual_income               0.009770
lead_score                number_of_courses_viewed   -0.004879
number_of_courses_viewed  lead_score                 -0.004879
                          interaction_count          -0.023565
interaction_count         number_of_courses_viewed   -0.023565
dtype: float64

#### The two features that have the biggest correlation are `Annual Income and Interaction Count`

### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value `y` is not in your dataframe.
  
### Question 3

* Calculate the mutual information score between `y` and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Which of these variables has the biggest mutual information score?
  
- `industry`
- `location`
- `lead_source`
- `employment_status`

In [29]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [31]:
len(df_train),len(df_val),len(df_test)

(876, 293, 293)

In [32]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test= df_test.reset_index(drop=True)

In [33]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [34]:
del df_train['converted']
del df_val['converted']
del df_test['converted']


In [38]:
def mutual_information_converted_score(series):
    return mutual_info_score(series, y_train)

In [62]:
df_train[categorical].apply(mutual_information_converted_score)

lead_source          0.035396
industry             0.011575
employment_status    0.012938
location             0.004464
dtype: float64

### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.64
- 0.74
- 0.84
- 0.94

In [58]:
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [71]:
train_dicts = df_train[categorical+numerical].to_dict(orient='records')

In [72]:
dv = DictVectorizer(sparse=False)

In [77]:
X_train = dv.fit_transform(train_dicts)

In [78]:
val_dicts = df_val[categorical+numerical].to_dict(orient='records')

In [80]:
X_val = dv.transform(val_dicts)
X_val[0]

array([5.222e+04, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 7.000e-02, 0.000e+00, 0.000e+00,
       0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       3.000e+00])

In [163]:
model = LogisticRegression(solver='liblinear', C=100.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,100.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [164]:
y_pred = model.predict(X_val)

In [165]:
accuracy_global_train = accuracy_score(y_val, y_pred)
accuracy_global_train

0.6996587030716723

#### The closest answer to the accuracy is ``

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model using the same features and parameters as in Q4 (without rounding).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- `'industry'`
- `'employment_status'`
- `'lead_score'`

> **Note**: The difference doesn't have to be positive.

In [102]:
categorical_no_industry = ['lead_source', 'employment_status', 'location']
categorical_no_employment_status = ['lead_source', 'industry', 'location']
categorical_no_lead_score = ['industry', 'employment_status', 'location']

#### No industry

In [104]:
train_dicts_no_industry = df_train[categorical_no_industry+numerical].to_dict(orient='records')

In [105]:
dv = DictVectorizer(sparse=False)

In [106]:
X_train_no_industry = dv.fit_transform(train_dicts_no_industry)

In [107]:
val_dicts_no_industry = df_val[categorical_no_industry+numerical].to_dict(orient='records')

In [108]:
X_val_no_industry = dv.transform(val_dicts_no_industry)
X_val_no_industry[0]

array([5.222e+04, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
       1.000e+00, 7.000e-02, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 3.000e+00])

In [109]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_no_industry, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [110]:
y_pred_no_industry = model.predict(X_val_no_industry)

In [112]:
accuracy_global_train_no_industry = accuracy_score(y_val, y_pred_no_industry)
accuracy_global_train_no_industry

0.6996587030716723

In [113]:
accuracy_global_train - accuracy_global_train_no_industry

0.0

#### No employment_status

In [114]:
train_dicts_no_employment_status = df_train[categorical_no_employment_status+numerical].to_dict(orient='records')

In [115]:
dv = DictVectorizer(sparse=False)

In [116]:
X_train_no_employment_status = dv.fit_transform(train_dicts_no_employment_status)

In [117]:
val_dicts_no_employment_status = df_val[categorical_no_employment_status+numerical].to_dict(orient='records')

In [118]:
X_val_no_employment_status = dv.transform(val_dicts_no_employment_status)
X_val_no_employment_status[0]

array([5.222e+04, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 7.000e-02, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 3.000e+00])

In [119]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_no_employment_status, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [120]:
y_pred_no_employment_status = model.predict(X_val_no_employment_status)

In [121]:
accuracy_global_train_no_employment_status = accuracy_score(y_val, y_pred_no_employment_status)
accuracy_global_train_no_employment_status

0.6962457337883959

In [122]:
accuracy_global_train - accuracy_global_train_no_employment_status

0.0034129692832763903

#### No lead_score

In [123]:
train_dicts_no_lead_score = df_train[categorical_no_lead_score+numerical].to_dict(orient='records')

In [124]:
dv = DictVectorizer(sparse=False)

In [125]:
X_train_no_lead_score = dv.fit_transform(train_dicts_no_lead_score)

In [126]:
val_dicts_no_lead_score = df_val[categorical_no_lead_score+numerical].to_dict(orient='records')

In [127]:
X_val_no_lead_score = dv.transform(val_dicts_no_lead_score)
X_val_no_lead_score[0]

array([5.222e+04, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 7.000e-02, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       3.000e+00])

In [128]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_no_lead_score, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [129]:
y_pred_no_lead_score = model.predict(X_val_no_lead_score)

In [130]:
accuracy_global_train_no_lead_score = accuracy_score(y_val, y_pred_no_lead_score)
accuracy_global_train_no_lead_score

0.7030716723549488

In [131]:
accuracy_global_train - accuracy_global_train_no_lead_score

-0.0034129692832765013

In [132]:
np.sort([accuracy_global_train - accuracy_global_train_no_industry, accuracy_global_train - accuracy_global_train_no_employment_status, accuracy_global_train - accuracy_global_train_no_lead_score])

array([-0.00341297,  0.        ,  0.00341297])

In [133]:
least_usefull_feature = {
    'industry': accuracy_global_train - accuracy_global_train_no_industry,
    'employment_status': accuracy_global_train - accuracy_global_train_no_employment_status,
    'lead_score': accuracy_global_train - accuracy_global_train_no_lead_score
    
}

In [134]:
least_usefull_feature

{'industry': 0.0,
 'employment_status': 0.0034129692832763903,
 'lead_score': -0.0034129692832765013}

#### The least usefull feature is `Industry`

### Question 6

* Now let's train a regularized logistic regression.
* Let's try the following values of the parameter `C`: `[0.01, 0.1, 1, 10, 100]`.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these `C` leads to the best accuracy on the validation set?

- 0.01
- 0.1
- 1
- 10
- 100

> **Note**: If there are multiple options, select the smallest `C`.

In [144]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [145]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [146]:
len(df_train),len(df_val),len(df_test)

(876, 293, 293)

In [147]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test= df_test.reset_index(drop=True)

In [148]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

In [149]:
del df_train['converted']
del df_val['converted']
del df_test['converted']


In [179]:
train_dicts = df_train[categorical+numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

for C in [0.01, 0.1, 1, 10, 100]:
    
    model = LogisticRegression(solver='liblinear', C=C, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    
    accuracy = accuracy_score(y_val, y_pred)
    print('C= %s Accuracy %s' % (C, round(accuracy,3)))

C= 0.01 Accuracy 0.7
C= 0.1 Accuracy 0.7
C= 1 Accuracy 0.7
C= 10 Accuracy 0.7
C= 100 Accuracy 0.7


#### All of the options got the same result, so the lowest one will be the answer that is `C = 0.01`